### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

### Compile contract from file

In [1]:
# Import libraries

import json
import web3
from datetime import datetime
from web3 import Web3
from solcx import compile_source
import solcx
#from web3.contract import ConciseContract

In [2]:
# Read in LUCE contract code
with open('./data/ConsentCode.sol', 'r') as file:
    contract_source_code = file.read()

In [3]:
# Compile & Store Compiled source code
solcx.install_solc('0.6.2')
compiled_sol = compile_source(contract_source_code,solc_version='0.6.2')

In [4]:
#compiled_sol

In [5]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:ConsentCode']

In [6]:
#contract_interface

In [7]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [8]:
startTime = datetime.now()

In [9]:
# Use Ganache for web3 instance
url = 'http://localhost:8545'

w3 = Web3(Web3.HTTPProvider(url))

In [10]:
# Use local Ganache GUI on macOS
#w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

In [11]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [12]:
w3.eth.accounts[0]

'0x84dF8A457982333C0893b968872F57b963243307'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [13]:
# Create contract blueprint
# from contract_abi_bytecode import abi, bytecode
Contract = w3.eth.contract(abi=abi, bytecode=bytecode)

In [14]:
# Submit the transaction that deploys the contract
tx_hash = Contract.constructor().transact({"from": w3.eth.accounts[0]})

In [15]:
tx_hash

HexBytes('0x355ae20ead2292db3cec36a4296bd941e6bc157238ce04a24758c52d11ba28fb')

### Obtain Transcation Receipt

In [16]:
# Wait for the transaction to be mined, and get the transaction receipt
# tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
import time

while True:
    try:
        tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
        if tx_receipt is not None:
            break  # Exit the loop if the receipt is available
    except ValueError:
        pass  # Handle the case when the transaction is not yet mined

    time.sleep(1)  # Wait for 1 second before checking again

# Now you can use tx_receipt in your further processing

In [17]:
accounts = w3.eth.accounts
print("Available accounts:", accounts)

Available accounts: ['0x84dF8A457982333C0893b968872F57b963243307', '0x159f33a7b5a9fB939b8AC6f8CE2E54e7eaE13bE0', '0x68C0eC5c56f0EDe43affA748e5D4aFd87f31513d']


In [18]:
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number",contract_block,".")

The contract is deployed with block number 2 .


In [19]:
# With obtain the final address of the contract 
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

The contract has the address 0xea36d1D59772835aD60A8adef0BE54e2cB6d5889 .


### Interact with contract

In [20]:
# Create python instance of deployed contract
Contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [21]:
# Create additional accounts
num_additional_accounts = 13
additional_accounts = [w3.eth.account.create() for _ in range(num_additional_accounts)]

# Print the addresses of additional accounts
for i, account in enumerate(additional_accounts, start=1):
    print(f"Additional Account {i} Address:", account.address)

# Add the addresses of additional accounts to the accounts variable
accounts += [account.address for account in additional_accounts]

print("Updated List of Accounts:", accounts)

# Now you can use the updated list of accounts for interacting with the contract

Additional Account 1 Address: 0x19008b4F18BBf5E5F3f238b302e784f67067A0AD
Additional Account 2 Address: 0x52D01d08F7D429c653c6Ffd27901FB30a3fd0295
Additional Account 3 Address: 0xd8F91BfDC7500681e001A8dFeab97dbC8D437208
Additional Account 4 Address: 0xf22532C47B211C4cAc573d23bBA12Ba1d659aD06
Additional Account 5 Address: 0x835f70863c50D4448BA50929FA294e9D8c27737c
Additional Account 6 Address: 0x1f522A705f34b0641762E42D8907268243ef67bB
Additional Account 7 Address: 0xb741E7fE0Ef5F448CbBe678405e3db47681CF466
Additional Account 8 Address: 0xcfe70f12af906A1f57DDdaA39a66053889509bf3
Additional Account 9 Address: 0x5054Fc6Fe675445de3514c3e86a4A62F8ba3D815
Additional Account 10 Address: 0x0B4594518D8fB92242365a63e65bb4F434F84169
Additional Account 11 Address: 0x3531751f370e58f422356ce500C481C59Ae9e3eA
Additional Account 12 Address: 0x73f811E62164e69eaf046Ed5e4D5307A58049585
Additional Account 13 Address: 0x401555425c908B1fB5EA74C1e5ab721dc2F71CB5
Updated List of Accounts: ['0x84dF8A457982333C0

In [22]:
# Extract default accounts created by ganache
# accounts = w3.eth.accounts

In [23]:
accounts

['0x84dF8A457982333C0893b968872F57b963243307',
 '0x159f33a7b5a9fB939b8AC6f8CE2E54e7eaE13bE0',
 '0x68C0eC5c56f0EDe43affA748e5D4aFd87f31513d',
 '0x19008b4F18BBf5E5F3f238b302e784f67067A0AD',
 '0x52D01d08F7D429c653c6Ffd27901FB30a3fd0295',
 '0xd8F91BfDC7500681e001A8dFeab97dbC8D437208',
 '0xf22532C47B211C4cAc573d23bBA12Ba1d659aD06',
 '0x835f70863c50D4448BA50929FA294e9D8c27737c',
 '0x1f522A705f34b0641762E42D8907268243ef67bB',
 '0xb741E7fE0Ef5F448CbBe678405e3db47681CF466',
 '0xcfe70f12af906A1f57DDdaA39a66053889509bf3',
 '0x5054Fc6Fe675445de3514c3e86a4A62F8ba3D815',
 '0x0B4594518D8fB92242365a63e65bb4F434F84169',
 '0x3531751f370e58f422356ce500C481C59Ae9e3eA',
 '0x73f811E62164e69eaf046Ed5e4D5307A58049585',
 '0x401555425c908B1fB5EA74C1e5ab721dc2F71CB5']

**UploadData**  
```solidity
   function UploadDataPrimaryCategory(
        address _address1,
        bool _NoRestrictions,
        bool _OpenToGeneralResearchAndClinicalCare,
        bool _OpenToHMBResearch,
        bool _OpenToPopulationAndAncestryResearch,
        bool _OpenToDiseaseSpecific)
```
```solidity
    function UploadDataSecondaryCategory(
        address _address1, 
        bool _OpenToGeneticStudiesOnly,
        bool _ResearchSpecificRestrictions,
        bool _OpenToResearchUseOnly,
        bool _NoGeneralMethodResearch)
```

```solidity
   function UploadDataRequirements(
        address _address1,
        bool _GeographicSpecificRestriction,
        bool _OpenToNonProfitUseOnly,
        bool _PublicationRequired,
        bool _CollaborationRequired,
        bool _EthicsApprovalrequired,
        bool _TimeLimitOnUse,
        bool _CostOnUse,
        bool _DataSecurityMeasuresRequired)
```

In [28]:
# Assuming you have already defined the Contract and test_address0 variables
upload_data_function = Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True)

# Set a custom gas limit (you may need to adjust this based on your contract's complexity)
custom_gas_limit = 200000

# Get the gas price
gas_price = w3.eth.gas_price

# Now you can execute the function call
tx_hash_upload = upload_data_function.transact({"from": w3.eth.accounts[0], "gas": custom_gas_limit, "gasPrice": gas_price})

In [26]:
#for i in range(100):
#    test_address0 = accounts[0]
#    tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).transact()
#    tx_hash = Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).transact()

In [25]:
#endTimeDataProvider10 = datetime.now()
#elapsedTimeDataProvider10 = endTimeDataProvider10 - startTime0
#print("elapsedTimeDataProvider10", elapsedTimeDataProvider10)

In [27]:
# Assuming you have already defined the Contract and accounts variables
test_address0 = accounts[0]

# Set custom gas limit (you may need to adjust this based on your contract's complexity)
custom_gas_limit = 200000

# Get the gas price
gas_price = w3.eth.gas_price

# Execute UploadDataPrimaryCategory function
tx_hash_primary_category = Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).transact({"from": test_address0, "gas": custom_gas_limit, "gasPrice": gas_price})

# Execute UploadDataRequirements function
tx_hash_requirements = Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).transact({"from": test_address0, "gas": custom_gas_limit, "gasPrice": gas_price})

In [2496]:
#print(w3.eth.getBlock('latest'))

In [2497]:
#endTimeDataProvider0 = datetime.now()
#elapsedTimeDataProvider0 = endTimeDataProvider0 - startTime0
#print("elapsedTimeDataProvider0", elapsedTimeDataProvider0)

In [2498]:
#tx_hash

**giveResearchPurpose**  
```solidity
 function giveResearchPurpose(
        address _address2,
        bool _UseForMethodsDevelopment,
        bool _UseForReferenceOrControlMaterial,
        bool _UseForPopulationsResearch,
        bool _UseForAncestryResearch,
        bool _UseForHMBResearch)
```

**giveHMBPurpose**  
```solidity
 function giveHMBPurpose( 
        address _address2, 
        bool _UseForFundamentalBioResearch,
        bool _UseForGeneticsResearch,
        bool _UseForDrugDevelopmentResearch, 
        bool _UseForAnyDiseaseResearch,
        bool _UseForAgeCategoriesResearch,
        bool _UseForGenderCategoriesResearch)
```

**giveClinicalPurpose**  
```solidity
 function giveClinicalPurpose( 
        address _address2, 
        bool _UseForDecisionSupport, 
        bool _UseForDiseaseSupport)
```

**givePerson**  
```solidity
 function givePerson( 
        address _address2,
        bool _UseByAcademicProfessionals,
        bool _UseByClinicalProfessionals,
        bool _UseByProfitMakingProfessionals, 
        bool _UseByNonProfessionals)
```

**giveGeographicSpecificRestriction**  
```solidity
 function giveGeographicSpecificRestriction( 
        address _address2, 
        bool _UseBySpecifiedCountries)
```
**giveProfit**  
```solidity
 function giveProfit( 
        address _address2, 
        bool _UseForProfitPurpose,
        bool _UseForNonProfitPurpose)
```
**giveDataRequester_Terms**  
```solidity
 function giveDataRequester_Terms( 
        address _address2, 
        bool _NoTimelineRestrictions,
        bool _NoFormalApprovalRequired,
        bool _NoCollaborationRequired,
        bool _NoPublicationRequired,
        bool _NoDataSecurityMeasures,
        bool _NoDataDestructionRequired,
        bool _NoLinkingOfAccessedRecords,
        bool _NoRecontactingDataSubjects,
        bool _NoIntellectualPropertyClaims,
        bool _NoUseOfAccessedResources,
        bool _NoFeesForAccess)
```

In [29]:
startTime13 = datetime.now()

In [30]:
#Data Requesters Account  
test_address13 = accounts[2]

#Data Requester 1 (General Research and Profit-Making)
tx_hash = Contract.functions.giveResearchPurpose(test_address13, True, True, False, False, False).transact({"from": w3.eth.accounts[2]})
tx_hash = Contract.functions.givePerson(test_address13, True, False, True, False).transact({"from": w3.eth.accounts[2]})
tx_hash = Contract.functions.giveProfit(test_address13, True, False).transact({"from": w3.eth.accounts[2]})

In [31]:
endTimeDataRequester13 = datetime.now()
elapsedTimeDataRequester13 = endTimeDataRequester13 - startTime13
print("elapsedTimeDataRequester13", elapsedTimeDataRequester13)

elapsedTimeDataRequester13 0:00:07.630836


In [32]:
Contract.functions.AccessData(test_address0, test_address13).call()

False

In [33]:
#endTimeDataRequesterAccessData13 = datetime.now()
#elapsedTimeDataRequesterAccessData13 = endTimeDataRequesterAccessData13 - startTime13
#print("elapsedTimeDataRequesterAccessData13", elapsedTimeDataRequesterAccessData13)

In [34]:
startTime14 = datetime.now()

In [37]:
#Data Requesters Account
test_address14 = accounts[14]
#Data Requester 2 (HMB and Profit Making)
tx_hash = Contract.functions.giveHMBPurpose(test_address14, True, True, True, False, True, True).transact()
tx_hash = Contract.functions.givePerson(test_address14, False, True, True, False).transact()
tx_hash = Contract.functions.giveProfit(test_address14, True, False).transact()

ValueError: {'message': 'from not found; is required', 'stack': 'Error: from not found; is required\n    at EthereumApi.eth_sendTransaction (/app/dist/node/1.js:2:87278)\n    at EthereumApi.n.value (/app/dist/node/1.js:2:238968)\n    at Object.execute (/app/dist/node/1.js:2:165607)\n    at RequestCoordinator.<anonymous> (/app/dist/node/1.js:2:165421)\n    at /app/dist/node/1.js:2:165687\n    at new Promise (<anonymous>)\n    at RequestCoordinator.queue (/app/dist/node/1.js:2:165529)\n    at Executor.execute (/app/dist/node/1.js:2:164094)\n    at EthereumProvider._requestRaw (/app/dist/node/1.js:2:130922)\n    at Connector.<anonymous> (/app/dist/node/1.js:2:125410)', 'code': -32700}

In [36]:
endTimeDataRequester14 = datetime.now()
elapsedTimeDataRequester14 = endTimeDataRequester14 - startTime14
print("elapsedTimeDataRequester14", elapsedTimeDataRequester14)

elapsedTimeDataRequester14 0:00:10.963842


In [38]:
Contract.functions.AccessData(test_address0, test_address14).call()

False

In [39]:
#endTimeDataRequesterAccessData14 = datetime.now()
#elapsedTimeDataRequesterAccessData14 = endTimeDataRequesterAccessData14 - startTime14
#print("elapsedTimeDataRequesterAccessData14", elapsedTimeDataRequesterAccessData14)

In [40]:
startTime15 = datetime.now()

In [41]:
#Data Requesters Account  
test_address15 = accounts[15]
#Data Requester 3 (Disease Specific and Non-Profit-Making)
tx_hash = Contract.functions.giveHMBPurpose(test_address15, False, False, False, True, False, False).transact()
tx_hash = Contract.functions.givePerson(test_address15, False, True, False, False).transact()
tx_hash = Contract.functions.giveProfit(test_address15, False, True).transact()

ValueError: {'message': 'from not found; is required', 'stack': 'Error: from not found; is required\n    at EthereumApi.eth_sendTransaction (/app/dist/node/1.js:2:87278)\n    at EthereumApi.n.value (/app/dist/node/1.js:2:238968)\n    at Object.execute (/app/dist/node/1.js:2:165607)\n    at RequestCoordinator.<anonymous> (/app/dist/node/1.js:2:165421)\n    at /app/dist/node/1.js:2:165687\n    at new Promise (<anonymous>)\n    at RequestCoordinator.queue (/app/dist/node/1.js:2:165529)\n    at Executor.execute (/app/dist/node/1.js:2:164094)\n    at EthereumProvider._requestRaw (/app/dist/node/1.js:2:130922)\n    at Connector.<anonymous> (/app/dist/node/1.js:2:125410)', 'code': -32700}

In [2511]:
endTimeDataRequester15 = datetime.now()
elapsedTimeDataRequester15 = endTimeDataRequester15 - startTime15
print("elapsedTimeDataRequester15", elapsedTimeDataRequester15)

elapsedTimeDataRequester15 0:00:00.223179


In [2512]:
Contract.functions.AccessData(test_address0, test_address15).call()

True

In [2513]:
endTimeDataRequesterAccessData15 = datetime.now()
elapsedTimeDataRequesterAccessData15 = endTimeDataRequesterAccessData15 - startTime15
print("elapsedTimeDataRequesterAccessData15", elapsedTimeDataRequesterAccessData15)

elapsedTimeDataRequesterAccessData15 0:00:00.286072


In [2514]:
startTime1 = datetime.now()

In [2515]:
#Data provider consent (General research & profit making)
#D-002
test_address1 = accounts[1]
tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address1, False, True, True, True, True).transact()

In [2516]:
endTimeDataProvider1 = datetime.now()
elapsedTimeDataProvider1 = endTimeDataProvider1 - startTime1
print("elapsedTimeDataProvider1", elapsedTimeDataProvider1)

elapsedTimeDataProvider1 0:00:00.083038


In [2517]:
startTime1_13 = datetime.now()

In [2518]:
Contract.functions.AccessData(test_address1, test_address13).call()

True

In [2519]:
endTimeDataRequesterAccessData1_13 = datetime.now()
elapsedTimeDataRequesterAccessData1_13 = endTimeDataRequesterAccessData1_13 - startTime1_13
print("elapsedTimeDataRequesterAccessData1_13", elapsedTimeDataRequesterAccessData1_13)

elapsedTimeDataRequesterAccessData1_13 0:00:00.053825


In [2520]:
startTime1_14 = datetime.now()

In [2521]:
Contract.functions.AccessData(test_address1, test_address14).call()

True

In [2522]:
endTimeDataRequesterAccessData1_14 = datetime.now()
elapsedTimeDataRequesterAccessData1_14 = endTimeDataRequesterAccessData1_14 - startTime1_14
print("elapsedTimeDataRequesterAccessData1_14", elapsedTimeDataRequesterAccessData1_14)

elapsedTimeDataRequesterAccessData1_14 0:00:00.044319


In [2523]:
startTime1_15 = datetime.now()

In [2524]:
Contract.functions.AccessData(test_address1, test_address15).call()

True

In [2525]:
endTimeDataRequesterAccessData1_15 = datetime.now()
elapsedTimeDataRequesterAccessData1_15 = endTimeDataRequesterAccessData1_15 - startTime1_15
print("elapsedTimeDataRequesterAccessData1_15", elapsedTimeDataRequesterAccessData1_15)

elapsedTimeDataRequesterAccessData1_15 0:00:00.049678


In [2526]:
#startTime2 = datetime.now()

In [2527]:
#Data provider consent (General research & non-profit making)
#H-001 
test_address2 = accounts[2]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address2, False, True, False, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address2, False, True, False, False, False, False, False, False).transact()

In [2528]:
#endTimeDataProvider2 = datetime.now()
#elapsedTimeDataProvider2 = endTimeDataProvider2 - startTime2
#print("elapsedTimeDataProvider2", elapsedTimeDataProvider2)

In [2529]:
#startTime2_13 = datetime.now()

In [2530]:
Contract.functions.AccessData(test_address2, test_address13).call()

False

In [2531]:
#endTimeDataRequesterAccessData2_13 = datetime.now()
#elapsedTimeDataRequesterAccessData2_13 = endTimeDataRequesterAccessData2_13 - startTime2_13
#print("elapsedTimeDataRequesterAccessData2_13", elapsedTimeDataRequesterAccessData2_13)

In [2532]:
#startTime2_14 = datetime.now()

In [2533]:
Contract.functions.AccessData(test_address2, test_address14).call()

False

In [2534]:
#endTimeDataRequesterAccessData2_14 = datetime.now()
#elapsedTimeDataRequesterAccessData2_14 = endTimeDataRequesterAccessData2_14 - startTime2_14
#print("elapsedTimeDataRequesterAccessData2_14", elapsedTimeDataRequesterAccessData2_14)

In [2535]:
#startTime2_15 = datetime.now()

In [2536]:
Contract.functions.AccessData(test_address2, test_address15).call()

False

In [2537]:
#endTimeDataRequesterAccessData2_15 = datetime.now()
#elapsedTimeDataRequesterAccessData2_15 = endTimeDataRequesterAccessData2_15 - startTime2_15
#print("elapsedTimeDataRequesterAccessData2_15", elapsedTimeDataRequesterAccessData2_15)#

In [2538]:
#startTime3 = datetime.now()

In [2539]:
#Data provider consent (General research & profit making)
#H-002 
test_address3 = accounts[3]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address3, False, True, False, False, False).transact()

In [2540]:
#endTimeDataProvider3 = datetime.now()
#elapsedTimeDataProvider3 = endTimeDataProvider3 - startTime3
#print("elapsedTimeDataProvider3", elapsedTimeDataProvider3)

In [2541]:
#startTime3_13 = datetime.now()

In [2542]:
Contract.functions.AccessData(test_address3, test_address13).call()

True

In [2543]:
#endTimeDataRequesterAccessData3_13 = datetime.now()
#elapsedTimeDataRequesterAccessData3_13 = endTimeDataRequesterAccessData3_13 - startTime3_13
#print("elapsedTimeDataRequesterAccessData3_13", elapsedTimeDataRequesterAccessData3_13)

In [2544]:
#startTime3_14 = datetime.now()

In [2545]:
Contract.functions.AccessData(test_address3, test_address14).call()

False

In [2546]:
#endTimeDataRequesterAccessData3_14 = datetime.now()
#elapsedTimeDataRequesterAccessData3_14 = endTimeDataRequesterAccessData3_14 - startTime3_14
#print("elapsedTimeDataRequesterAccessData3_14", elapsedTimeDataRequesterAccessData3_14)

In [2547]:
#startTime3_15 = datetime.now()

In [2548]:
Contract.functions.AccessData(test_address3, test_address15).call()

False

In [2549]:
#endTimeDataRequesterAccessData3_15 = datetime.now()
#elapsedTimeDataRequesterAccessData3_15 = endTimeDataRequesterAccessData3_15 - startTime3_15
#print("elapsedTimeDataRequesterAccessData3_15", elapsedTimeDataRequesterAccessData3_15)

In [2550]:
#startTime4 = datetime.now()

In [2551]:
#Data provider consent (HMB research & non-profit making)
#D-003 
test_address4 = accounts[4]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address4, False, False, True, False, True).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address4, False, True, False, False, False, False, False, False).transact()

In [2552]:
#endTimeDataProvider4 = datetime.now()
#elapsedTimeDataProvider4 = endTimeDataProvider4 - startTime4
#print("elapsedTimeDataProvider4", elapsedTimeDataProvider4)

In [2553]:
#startTime4_13 = datetime.now()

In [2554]:
Contract.functions.AccessData(test_address4, test_address13).call()

False

In [2555]:
#endTimeDataRequesterAccessData4_13 = datetime.now()
#elapsedTimeDataRequesterAccessData4_13 = endTimeDataRequesterAccessData4_13 - startTime4_13
#print("elapsedTimeDataRequesterAccessData4_13", elapsedTimeDataRequesterAccessData4_13)

In [2556]:
#startTime4_14 = datetime.now()

In [2557]:
Contract.functions.AccessData(test_address4, test_address14).call()

False

In [2558]:
#endTimeDataRequesterAccessData4_14 = datetime.now()
#elapsedTimeDataRequesterAccessData4_14 = endTimeDataRequesterAccessData4_14 - startTime4_14
#print("elapsedTimeDataRequesterAccessData4_14", elapsedTimeDataRequesterAccessData4_14)

In [2559]:
#startTime4_15 = datetime.now()

In [2560]:
Contract.functions.AccessData(test_address4, test_address15).call()

True

In [2561]:
#endTimeDataRequesterAccessData4_15 = datetime.now()
#elapsedTimeDataRequesterAccessData4_15 = endTimeDataRequesterAccessData4_15 - startTime4_15
#print("elapsedTimeDataRequesterAccessData4_15", elapsedTimeDataRequesterAccessData4_15)

In [2562]:
startTime5 = datetime.now()

In [2563]:
#Data provider consent (HMB research & profit making)
#D-004 
test_address5 = accounts[5]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address5, False, False, True, False, True).transact()

In [2564]:
endTimeDataProvider5 = datetime.now()
elapsedTimeDataProvider5 = endTimeDataProvider5 - startTime5
print("elapsedTimeDataProvider5", elapsedTimeDataProvider5)

elapsedTimeDataProvider5 0:00:00.068086


In [2565]:
startTime5_13 = datetime.now()

In [2566]:
Contract.functions.AccessData(test_address5, test_address13).call()

False

In [2567]:
endTimeDataRequesterAccessData5_13 = datetime.now()
elapsedTimeDataRequesterAccessData5_13 = endTimeDataRequesterAccessData5_13 - startTime5_13
print("elapsedTimeDataRequesterAccessData5_13", elapsedTimeDataRequesterAccessData5_13)

elapsedTimeDataRequesterAccessData5_13 0:00:00.030761


In [2568]:
startTime5_14 = datetime.now()

In [2569]:
Contract.functions.AccessData(test_address5, test_address14).call()

True

In [2570]:
endTimeDataRequesterAccessData5_14 = datetime.now()
elapsedTimeDataRequesterAccessData5_14 = endTimeDataRequesterAccessData5_14 - startTime5_14
print("elapsedTimeDataRequesterAccessData5_14", elapsedTimeDataRequesterAccessData5_14)

elapsedTimeDataRequesterAccessData5_14 0:00:00.049857


In [2571]:
startTime5_15 = datetime.now()

In [2572]:
Contract.functions.AccessData(test_address5, test_address15).call()

True

In [2573]:
endTimeDataRequesterAccessData5_15 = datetime.now()
elapsedTimeDataRequesterAccessData5_15 = endTimeDataRequesterAccessData5_15 - startTime5_15
print("elapsedTimeDataRequesterAccessData5_15", elapsedTimeDataRequesterAccessData5_15)

elapsedTimeDataRequesterAccessData5_15 0:00:00.043902


In [2574]:
#Data provider consent (HMB research & non-profit making)
#H-003 
test_address6 = accounts[6]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address6, False, False, True, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address6, False, True, False, False, False, False, False, False).transact()

In [2575]:
Contract.functions.AccessData(test_address6, test_address13).call()

False

In [2576]:
Contract.functions.AccessData(test_address6, test_address14).call()

False

In [2577]:
Contract.functions.AccessData(test_address6, test_address15).call()

False

In [2578]:
#Data provider consent (HMB research & profit making)
#H-004 
test_address7 = accounts[7]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address7, False, False, True, False, False).transact()

In [2579]:
Contract.functions.AccessData(test_address7, test_address13).call()

False

In [2580]:
Contract.functions.AccessData(test_address7, test_address14).call()

True

In [2581]:
Contract.functions.AccessData(test_address7, test_address15).call()

False

In [2582]:
startTime8 = datetime.now()

In [2583]:
#Data provider consent (Disease-specific research & non-profit making)
#D-005 
test_address8 = accounts[8]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address8, False, False, False, False, True).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address8, False, True, False, False, False, False, False, False).transact()

In [2584]:
endTimeDataProvider8 = datetime.now()
elapsedTimeDataProvider8 = endTimeDataProvider8 - startTime8
print("elapsedTimeDataProvider8", elapsedTimeDataProvider8)

elapsedTimeDataProvider8 0:00:00.150563


In [2585]:
startTime8_13 = datetime.now()

In [2586]:
Contract.functions.AccessData(test_address8, test_address13).call()

False

In [2587]:
endTimeDataRequesterAccessData8_13 = datetime.now()
elapsedTimeDataRequesterAccessData8_13 = endTimeDataRequesterAccessData8_13 - startTime8_13
print("elapsedTimeDataRequesterAccessData8_13", elapsedTimeDataRequesterAccessData8_13)

elapsedTimeDataRequesterAccessData8_13 0:00:00.028084


In [2588]:
startTime8_14 = datetime.now()

In [2589]:
Contract.functions.AccessData(test_address8, test_address14).call()

False

In [2590]:
endTimeDataRequesterAccessData8_14 = datetime.now()
elapsedTimeDataRequesterAccessData8_14 = endTimeDataRequesterAccessData8_14 - startTime8_14
print("elapsedTimeDataRequesterAccessData8_14", elapsedTimeDataRequesterAccessData8_14)

elapsedTimeDataRequesterAccessData8_14 0:00:00.043044


In [2591]:
startTime8_15 = datetime.now()

In [2592]:
Contract.functions.AccessData(test_address8, test_address15).call()

True

In [2593]:
endTimeDataRequesterAccessData8_15 = datetime.now()
elapsedTimeDataRequesterAccessData8_15 = endTimeDataRequesterAccessData8_15 - startTime8_15
print("elapsedTimeDataRequesterAccessData8_15", elapsedTimeDataRequesterAccessData8_15)

elapsedTimeDataRequesterAccessData8_15 0:00:00.053568


In [2594]:
#Data provider consent (Disease-specific research & profit making)
# D-006 
test_address9 = accounts[9]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address9, False, False, False, False, True).transact()

In [2595]:
Contract.functions.AccessData(test_address9, test_address13).call()

False

In [2596]:
Contract.functions.AccessData(test_address9, test_address14).call()

False

In [2597]:
Contract.functions.AccessData(test_address9, test_address15).call()

True

In [2598]:
#Data provider consent (Disease-specific research & non-profit making)
#H-005 
test_address10 = accounts[10]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address10, False, False, False, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address10, False, True, False, False, False, False, False, False).transact()

In [2599]:
Contract.functions.AccessData(test_address10, test_address13).call()

False

In [2600]:
Contract.functions.AccessData(test_address10, test_address14).call()

False

In [2601]:
Contract.functions.AccessData(test_address10, test_address15).call()

False

In [2602]:
#Data provider consent (Disease-specific research & profit making)
#H-006 
test_address11 = accounts[11]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address11, False, False, False, False, False).transact()

In [2603]:
Contract.functions.AccessData(test_address11, test_address13).call()

False

In [2604]:
Contract.functions.AccessData(test_address11, test_address14).call()

False

In [2605]:
Contract.functions.AccessData(test_address11, test_address15).call()

False

In [2606]:
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)

elapsedTime1 0:00:03.706441


In [2607]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address8, False, False, False, False, True).estimateGas())
#print("Gas", Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).estimateGas())
#print("Gas", Contract.functions.giveResearchPurpose(test_address13, True, True, False, False, False).estimateGas())
#print("Gas", Contract.functions.givePerson(test_address13, True, False, True, False).estimateGas())
#print("Gas", Contract.functions.giveProfit(test_address13, True, False).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address13).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address14).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address15).estimateGas())